In [1020]:
import numpy as np

### Initialization

In [1113]:
''' Grid basis setup. '''

# Define grid point parameters
x_min, x_max = [0, 4]
z_min, z_max = [0, 4]
dx, dz = 0.2, 0.2
# Define basis "vectors" (bv) to outline grid formation (step size added to maximum bound to include it)
bv_x = np.arange(x_min, x_max+dx, dx)
bv_z = np.arange(z_min, z_max+dz, dz)
# Build base grid meshgrid
base_x, base_z = np.meshgrid(bv_x, bv_z)

# CFL number
cfl = 0.05
# Timestep (assume c = 1)
dt = cfl*dx
# Maximum time
t_max = 3
# Create time array
times = np.arange(0, t_max+dt, dt)

''' Initialize field grids. '''

# Define starter grid with dimensions of (X x Z x t)
base_grid = np.full(shape=(len(times), len(bv_x), len(bv_z)), fill_value=0, dtype=float)

# Define dynamic fields
data = {}
# Define fields of interest
field_names = ['x', 'z', 'u', 'w', 'p', 'b']
# Construct dictionary with initial values for each field
data = {key: base_grid if key not in ['x', 'z'] 
        else base_x if key in ['x'] 
        else base_z for key in field_names}

''' Constants. '''
# Reference density (rho_0)
rho_0 = 1
# Gravitational acceleration (m s^-2)
g = 1

# Define constants
constants = [rho_0, g]

''' Perturbation at center. '''
# Get center indices
center_j, center_i = len(bv_z) // 2, len(bv_x) // 2
# Size of perturbation
size = 1
# Perturb the pressure
# data['p'][0, 5, 5] = -0.01

# Run the model
for n in range(0, len(times)-1):
    data = main(data, constants, n, dt)

### Animation

In [1111]:
from matplotlib import animation
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable as ml
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
plt.rcParams["animation.html"] = "jshtml"
plt.ioff()
plt.cla()

fig, ax = plt.subplots(figsize=(5, 3))

# Colorbar
cax_width = 0.25*ax.get_position().width
divider = ml(ax)
cax = divider.append_axes('right', size=cax_width, pad=0.1)

skip = 4

# Animation function
def animate(frame):
    
    ax.cla()
    cax.cla()
    
    i = int(frame*skip)

    vmin, vmax = data['p'][i, :, :].min(), data['p'][i, :, :].max()
    # vmin, vmax = -0.01, 0.06
    # im = ax.pcolormesh(data['x'], data['z'], data['p'][i, :-1, :-1], vmin=vmin, vmax=vmax)
    if vmin < 0 and vmax > 0:
        norm = matplotlib.colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    else:
        norm=None
    im = ax.contourf(data['x'][:-1, :-1], data['z'][:-1, :-1], data['p'][i, :-1, :-1], levels=24, norm=norm, cmap='RdBu_r')
    cb = fig.colorbar(im, cax=cax)
    # q = ax.quiver(data['x'][:, :-1], data['z'][:-1, :-1], data['u'][i, :-1, :-1], data['w'][i, :-1, :-1])
    
    # Tracer
    # l, k = 10, 10
    # ax.scatter(data['x'][l, k] + 1e4*dt*data['u'][i, l, k], data['z'][l, k] + 1e4*dt*data['w'][i, l, k], c='r')
    
    # Metadata
    ax_title_str = r'Time: {0:.2f} s'.format(times[i])
    ax.set_title(label=ax_title_str)
    ax.set_aspect('equal')

# Animation generation and save settings
animation.FuncAnimation(fig, animate, frames=(data['u'].shape[0]-1) // skip, interval=50)

/opt/homebrew/Caskroom/miniforge/base/envs/dev/lib/python3.10/site-packages/matplotlib/contour.py:1479: UserWarning: Warning: converting a masked element to nan.
  self.zmax = float(z.max())
/opt/homebrew/Caskroom/miniforge/base/envs/dev/lib/python3.10/site-packages/matplotlib/contour.py:1480: UserWarning: Warning: converting a masked element to nan.
  self.zmin = float(z.min())


### Model skeleton

In [1112]:
def main(data, constants, n, dt):
    
    # Initialize basis vectors for iteration
    x, z = data['x'][0, :], data['z'][:, 0]
    
    # Initialize right-hand side holding Poisson values
    b = np.full(shape=(len(z)*len(x)), fill_value=0, dtype=float)
    
    # Define start and end indices for each axis
    start_x, end_x = 0, len(x)-1
    start_z, end_z = 0, len(z)-1
    
    # Iterate over grid
    for j in range(start_z, end_z):
        for i in range(start_x, end_x):
            
            # print('Step: {0} | Position: ({1}, {2}) | Reference values - start_z: {3}; end_z: {4}'.format(n, j, i, start_z, end_z))
            
            ''' Current time-stepping scheme: forward Euler. '''
            # Horizontal velocity equation
            data['u'][n+1, j, i] = data['u'][n, j, i] + dt*temp_disc(fxn_u(data, j, i, n, constants), dt, method='rk4')
    
            # Vertical velocity equation
            data['w'][n+1, j, i] = data['w'][n, j, i] + dt*temp_disc(fxn_w(data, j, i, n, constants), dt, method='rk4')
            
            ''' Fill Poisson RHS matrix. '''
            # Use boolean to control whether BC value added to RHS if at top or bottom boundary
            bool_bc = 1 if (j == start_z) or (j == end_z-1) else 0
            # Initialize BC value
            bc_value = 0
            # Assign BC value
            if bool_bc:
                if j == start_z:
                    bc_value = 0.0001 if n == 0 and (end_x//2-2 < i < end_x//2+2) else 0
                elif j == end_z-1:
                    bc_value = 0
                
            # print('({1}, {2}) | BC forcing value: {0}'.format(bool_bc*bc_value, j, i))
            
            # Populate Poisson RHS array
            b[end_z*j+i] = p_rhs(data, j, i, n, (end_z - start_z), (end_x - start_x), constants) + bool_bc*bc_value
            # print('({0}, {1}) | {2} | RHS: {2:.3e}'.format(j, i, b[end_z*j+i]))
            
    A = poisson_ms(data['p'][n])
    p = np.linalg.solve(A, b).reshape(len(z), len(x))
    
    # Iterate over grid
    for l in range(start_z, end_z):
        for k in range(start_x, end_x):
            # data['p'][n+1, l, k] = data['p'][n, l, k] + dt*p[l, k]
            data['p'][n+1, l, k] = data['p'][n, l, k] + dt*temp_disc(p[l, k], dt, method='rk4')
    
    return data

### Functions

#### Horizontal velocity equation

In [1018]:
def fxn_u(data, j, i, n, constants):
    rho_0, g = constants
    return (-data['u'][n, j, i]*d1(data, 'u', 'x', j, i) - 
            data['w'][n, j, i]*d1(data, 'u', 'z', j, i) - 
            (1/rho_0)*data['p'][n, j, i]*d1(data, 'p', 'x', j, i))

#### Vertical velocity equation

In [1017]:
def fxn_w(data, j, i, n, constants):
    rho_0, g = constants
    return (-data['u'][n, j, i]*d1(data, 'w', 'x', j, i) - 
            data['w'][n, j, i]*d1(data, 'w', 'z', j, i) - 
            (1/rho_0)*data['p'][n, j, i]*d1(data, 'p', 'z', j, i) + 
            data['b'][n, j, i])

#### Poisson grid left-hand-side generator (periodic in the horizontal, Dirichlet conditions in the vertical)

In [1016]:
def poisson_ms(F):
    ''' Poisson matrix system generator for a doubly-periodic 2D domain. '''

    # Input grid (this is the field you're analyzing for, typically pressure)
    # F = np.zeros(shape=(5, 5)) 
    # Get dimensions (height and width) of the grid
    h, w = F.shape
    # Initialize Poisson LHS grid (this is the 'A' in the matrix system 'Ax = b')
    A = np.full(shape=(h*w, h*w), fill_value=0)

    for j in range(0, h):
        for i in range(0, w):
            # Create empty array for the point at (j, i)
            arr = np.full(shape=(h*w), fill_value=0)
            # Calculate array index (in other words, what element number are we looking at)
            m = j*h + i
            # Get left, right, upper, and lower indices
            lt, rt, up, dn = (i-1) % w + j*h, (i+1) % w + j*h, ((j+1) * h + i) % (w*h), ((j-1) * h + i) % (w*h)
            # Assign weights
            if j == 0:
                arr[m], arr[lt], arr[rt], arr[up], arr[dn] = -4, 1, 1, 1, 0
            elif j == h-1:
                arr[m], arr[lt], arr[rt], arr[up], arr[dn] = -4, 1, 1, 0, 1
            else:
                arr[m], arr[lt], arr[rt], arr[up], arr[dn] = -4, 1, 1, 1, 1
            # Pop the array into the m-th row of the Poisson LHS grid
            A[m] = arr

    return A

#### RHS of Poisson equation dynamic pressure 

In [1015]:
def p_rhs(data, j, i, n, h, w, constants):
    rho_0, g = constants
    
    ''' Calculate the right-hand side (b). '''
    # Horizontal component
    d2p_x = (d1(data, 'u', 'x', j, i)**2 + 
             data['u'][n, j, i]*d2(data, 'u', 'z', j, i) + 
             d1(data, 'w', 'x', j, i)*d1(data, 'u', 'z', j, i) +
             data['w'][n, j, i]*md2(data, 'u', j, i))
    # Vertical component
    d2p_z = (d1(data, 'u', 'z', j, i)*d1(data, 'w', 'x', j, i) + 
             data['u'][n, j, i]*md2(data, 'w', j, i) + 
             d1(data, 'w', 'z', j, i)**2 +
             data['w'][n, j, i]*d2(data, 'w', 'z', j, i) - 
             d1(data, 'b', 'z', j, i))
    # Combine
    d2p = -rho_0 * (d2p_x + d2p_z)
    
    return d2p

#### Define spatial differentiation discretizations
The idea here is to generalize differentiation with a function for an order of differentiation. In other words, there will be one function for $\partial / \partial (n)$, one for $\partial^2 / \partial (n^2)$, etc.

##### First-order partial derivative

In [1014]:
def d1(data, var_name, dim_name, j, i, method='cdf2'):
    ''' First-order differentiation of a field.
    
    Inputs:
    - data:     the data dictionary
    - var_name: string with field name used for indexing 'data'
    - dim_name: dimension over which differentiation occurs
    - j, i:     index in the 2D grid
    - method:   differentiation scheme (default: second-order centered-difference, 'cdf2')
    '''
    
    # Define working data
    var = data[var_name][n, :, :]
    # Define working dimension
    dim = data[dim_name]
    # Initialize result
    res = np.nan
    
    if method == 'cdf2':
        if dim_name == 'x':
            res = (var[j, i+1] - var[j, i-1])/(2*(dim[j, i+1] - dim[j, i]))
        elif dim_name == 'z':
            res = (var[j+1, i] - var[j-1, i])/(2*(dim[j+1, i] - dim[j, i]))
    
    return res

##### Second-order partial derivative

In [1013]:
def d2(data, var_name, dim_name, j, i, method='cdf2'):
    ''' Second-order differentiation of a field.
    
    Inputs:
    - data:     the data dictionary
    - var_name: string with field name used for indexing 'data'
    - dim_name: dimension over which differentiation occurs
    - j, i:     index in the 2D grid
    - method:   differentiation scheme (default: second-order centered-difference, 'cdf2')
    '''
    
    # Define working data
    var = data[var_name][n, :, :]
    # Define working dimension
    dim = data[dim_name]
    # Initialize result
    res = np.nan
    
    if method == 'cdf2':
        if dim_name == 'x':
            res = (var[j, i+1] - 2*var[j, i] + var[j, i-1])/(dim[j, i+1] - dim[j, i])**2
        elif dim_name == 'z':
            res = (var[j+1, i] - 2*var[j, i] + var[j-1, i])/(dim[j+1, i] - dim[j, i])**2
            
    return res

##### First-order partial derivative

In [1012]:
def md2(data, var_name, j, i, method='cdf2'):
    ''' Second-order differentiation for mixed partials.
    
    Inputs:
    - data:     the data dictionary
    - var_name: string with field name used for indexing 'data'
    - j, i:     index in the 2D grid
    - method:   differentiation scheme (default: second-order centered-difference, 'cdf2')
    '''
    
    # Define working data
    var = data[var_name][n, :, :]
    # Initialize result
    res = np.nan
    
    if method == 'cdf2':
        res = (var[j+1, i+1] - var[j+1, i-1] - var[j-1, i+1] - var[j-1, i-1])/(4*(data['z'][j+1, i] - data['z'][j, i])*(data['x'][j, i+1] - data['x'][j, i]))
        
    return res

#### Define temporal differentiation discretizations.

In [1011]:
def temp_disc(value, dt, method='rk4'):
    # Initialize result
    res = np.nan
    
    if method == 'rk4':
        k1 = value
        k2 = value + (dt/2)*k1
        k3 = value + (dt/2)*k2
        k4 = value + (dt)*k3
        res = (k1 + 2*k2 + 2*k3 + k4)/6
        
    return res